In [8]:
import numpy
import re
import random
import sys
from collections import Counter
import string

def mention_(word):
    temp = word
    word = re.sub("@(\w+)", " <MENTION>" , word)
    if temp == word:
        return False, temp
    return True, word

def dupl_punc_(word):
    punc_set = set(string.punctuation)
    new_word = ''
    last = '4'
    for letter in word:
        if letter in punc_set:
            if last != letter:
                new_word += letter
                last = letter
        else:
            last = letter

            new_word += letter
    return new_word

def url_(word):
    var = word
    word = re.sub(r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)", " <URL>", word)
    if var == word:
        return False, var 
    return True , word

def hashtag_(word):
    temp = word
    word = re.sub("#(\w+)", " <HASHTAG>", word)
    if temp == word:
        return False, temp
    return True, word


def split_lines(line,n,fl=True):
    words = line.split(" ")
    tokens = []
    for word in words:
        tok =''
        ret, tok = url_(word)
        if ret == False:
            ret,tok = hashtag_(tok)
            ret,tok = mention_(tok)
            new_word = dupl_punc_(tok)
            new_word = re.sub('\'', " '", new_word)
            new_word = new_word.split()

            for w in new_word:
                tokens.append(w)
        else:
            tokens.append(tok)
    if fl==False:
        return ' '.join(tokens)
    else:
        for i in range(n-1):
            tokens.insert(0, '<START>')
            tokens.insert(len(tokens), '<END>')
        return " ".join(tokens)
        


def tokenize(file_p,n,fl=True):
    # lines.append("#ieroween THE STORY OF IEROWEEN! THE VIDEO -»»»»»»»»»»» http://bit.ly/2VFPAV «« JUST FOR FRANK !!! ÃƒÂ§")
    lines = []
    
    with open(file_p) as f:
        lines = f.readlines()
    tokens = ""
    for line in lines:
        tokens += split_lines(line[:-1],n,fl) + "\n"
    # print(tokens)
    return tokens



In [9]:


tok = tokenize("./data/general-tweets.txt",4,False)
with open("2019101095_tokenize.txt","w") as file:
    file.write(tok)
# print(tok)

In [79]:
ngrams_c = {}
ngram = []
unigram = []
bigram = []
trigram = []

def find_val(gram):
    if len(gram) == 0:
        return sum(ngrams_c[1].values())
    if gram in ngrams_c[len(gram)].keys():
        return ngrams_c[len(gram)][gram] 
    else:
        return 0

def witten2(gram):
    unique = 0
    for c in ngrams_c[len(gram)+1].keys():
        if(c[:-1] == gram):
            
            unique += 1
    k = float(unique + int(find_val(gram)))
    if(k != 0):
        return (unique / k)
    else:
        return random.uniform(0.00001,0.0001)

def witten(gram):
    x = len(gram)
    if x == 1:
        return find_val(gram) / sum(ngrams_c[1].values())
    if(find_val(gram[:-1]) != 0):
        return ((1 - witten2(gram[:-1])) * (find_val(gram) / find_val(gram[:-1]))) + (witten2(gram[:-1]) * witten(gram[:-1]))
    else:
        return random.uniform(0.00001, 0.0001) +  (witten2(gram[:-1]) * witten(gram[:-1]))
    
def kneser(gram, maxi):
    x = len(gram)
    d = 0.75
    if find_val(gram[:-1]) == 0:
        lamda = random.uniform(0,1)
        term = random.uniform(0.00001,0.0001)
    else:
        if x == maxi:
            term = max(0,find_val(gram)-d)/find_val(gram[:-1])
        else:
            term = max(0,(sum(token[1:] == gram for token in ngrams_c[len(gram) + 1].keys()) - d)) / find_val(gram[:-1])
        lamda = d * sum(token[:-1] == gram[:-1] for token in ngrams_c[len(gram)].keys()) / find_val(gram[:-1])
    if x == 1:
       return term
    else:
        return term + lamda * kneser(gram[:-1], maxi)

In [33]:
tokens = (split_lines("flvnfkjeniu delkjeriu euhegu heg",4)).split()
ooo=[]
for w in range(0,len(tokens)-1):
    ooo.append((" ".join(tokens[w:w+2])))
tokens = (split_lines("lakebklj wkjc j deg",4)).split()
for w in range(0,len(tokens)-1):
    ooo.append((" ".join(tokens[w:w+2])))
cc=Counter(ooo) 
cc 

Counter({'<START> <START>': 4,
         '<START> flvnfkjeniu': 1,
         'flvnfkjeniu delkjeriu': 1,
         'delkjeriu euhegu': 1,
         'euhegu heg': 1,
         'heg <END>': 1,
         '<END> <END>': 4,
         '<START> lakebklj': 1,
         'lakebklj wkjc': 1,
         'wkjc j': 1,
         'j deg': 1,
         'deg <END>': 1})

In [45]:
n = 4
ngrams_c = {}
ngram = []
unigram = []
bigram = []
trigram = []  
with open("./data/europarl-corpus.txt") as f:
    lines2 = f.readlines()
    random.shuffle(lines2)
    training = lines2

    for line in training:
        tokens = (split_lines(line,n)).split()
        for w in range(0,len(tokens)):
            unigram.append(("".join(tokens[w])))
    ngrams_c[1] = Counter(unigram)  

    for line in training:
        tokens = (split_lines(line,n)).split()
        for w in range(0,len(tokens)-1):
            bigram.append(("".join(tokens[w:w+2])))
    ngrams_c[2] = Counter(bigram)

    for line in training:
        tokens = (split_lines(line,n)).split()
        for w in range(0,len(tokens)-2):
            trigram.append(("".join(tokens[w:w+3])))
    ngrams_c[3] = Counter(trigram) 

    for line in training:
        tokens = (split_lines(line,n)).split()
        for w in range(0,len(tokens)-n+1):
            ngram.append(("".join(tokens[w:w+n])))
    ngrams_c[4] = Counter(ngram)   


In [46]:
test=[]
# test.append(input())
test.append("lekvkjrb oiv vfke d")

In [54]:
t='w'

In [70]:
i = 0
for line in test:
    inp =split_lines(line,4)
    words = inp.split()
    tex = []
    ngram_inp = []
    for x in range(0,len(words)-n+1):
        tex.append((" ".join(words[x:x+n])))
    ngram_inp = Counter(tex)

    # N=len(ngram_inp)
    # if t=='w' and True:
    #     p = 1
    #     for gram in ngram_inp:
    #         gram=tuple(gram.split())
    #         pp = ( 1.0 / witten(gram))**(1.0/N)
    #         p = p*pp
    #     print(p)  
    # else:
    #     p = 1
    #     for gram in ngram_inp:
    #         pp = ( 1.0 / kneser(gram,len(gram)))**(1.0/N)
    #         p = p*pp
    #     print(p)  


In [71]:
ngram_inp

Counter({'<START> <START> <START> lekvkjrb': 1,
         '<START> <START> lekvkjrb oiv': 1,
         '<START> lekvkjrb oiv vfke': 1,
         'lekvkjrb oiv vfke d': 1,
         'oiv vfke d <END>': 1,
         'vfke d <END> <END>': 1,
         'd <END> <END> <END>': 1})

In [81]:
for gram in ngram_inp:
    print(tuple(gram))
    pp = ( 1.0 / witten(gram))**(1.0/1)
    p = p*pp

('<', 'S', 'T', 'A', 'R', 'T', '>', ' ', '<', 'S', 'T', 'A', 'R', 'T', '>', ' ', '<', 'S', 'T', 'A', 'R', 'T', '>', ' ', 'l', 'e', 'k', 'v', 'k', 'j', 'r', 'b')


KeyError: 31

In [66]:
val="<START> <START> <START> lekvkjrb"

In [67]:
val=tuple(val.split())
val

('<START>', '<START>', '<START>', 'lekvkjrb')

In [82]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/bhavya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [83]:
val="kjre tht jgjt jg lwkjh gjkgh j"

In [84]:
nltk.word_tokenize(val)

['kjre', 'tht', 'jgjt', 'jg', 'lwkjh', 'gjkgh', 'j']

In [86]:
split_lines(val,n,False).split()

['kjre', 'tht', 'jgjt', 'jg', 'lwkjh', 'gjkgh', 'j']